In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime

# 데이터 불러오기

In [ ]:
sentence = pd.read_excel('/content/drive/MyDrive/aiContest/20201015_경진대회데이터_문장의미태깅.xlsx', sheet_name='문장f')
sentence.head()

,논문고유번호,태그,문장,대표여부
0,JAKO201003949970469,문제 정의,재난관리체계의 개선(일원화)방법으로는 여러 가지\r\n가 있을 수 있겠으나 여기서는...,Y
1,JAKO201003949970469,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간\r\n의 얼마만큼 재난관리와 관련...,N
2,JAKO201003949970469,대상 데이터,본 연구의 사용된 표본의 특성은 Table 5와 같다. 설\r\n문지 응답자들의 근...,N
3,JAKO201003949970469,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요\r\n인을 다중응답 분석을 이용해서...,N
4,JAKO201003949970469,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구\r\n목적을 달성하기 위하여 소방방재...,Y


# 결측값 처리

In [ ]:
# 문장 '없음' 인덱스 리스트로 생성
non_list = list(df[df['문장']=='없음'].index)+list(df[df['문장'].str.get(-2)=='없'].index)
# 제거 전 행 개수 출력
print(len(df))
# 값이 없는 행 제거
df = df.drop([13229, 13230, 13237, 13244]+non_list, axis=0)
print(len(df))
# 중복제거
df.drop_duplicates(subset = ['태그','문장'], inplace=True)
print(len(df))

# 컬럼 / 문장 전처리

In [ ]:
# 필요없는 열 제거
sentence = sentence.drop(['논문고유번호', '대표여부'], axis=1)
sentence.head()

,태그,문장
0,문제 정의,재난관리체계의 개선(일원화)방법으로는 여러 가지\r\n가 있을 수 있겠으나 여기서는...
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간\r\n의 얼마만큼 재난관리와 관련...
2,대상 데이터,본 연구의 사용된 표본의 특성은 Table 5와 같다. 설\r\n문지 응답자들의 근...
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요\r\n인을 다중응답 분석을 이용해서...
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구\r\n목적을 달성하기 위하여 소방방재...


In [ ]:
# 필요없는 문자 제거
sentence = sentence.replace({'\r':'','\n':'','\([^)]*\)' : ''}, regex=True)
sentence['문장'] = sentence['문장'].str.upper()
sentence['문장'] = sentence['문장'].str.replace("[^가-힣A-Z0-9=%\. ]","")
sentence.head()

,태그,문장,tokenized_nn
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...,"[재난, 관리, 체계, 개선, 방법, 가지, 재난, 관리, 분야, 사실, 중추적, ..."
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...,"[연구, 재난, 관리, 조직, 부처, 간, 얼마, 재난, 관리, 관련, 공조, 체계..."
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...,"[연구, 표본, 특성, 설문지, 응답자, 근무처, 소방, 방재, 청, 지방자치단체]"
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...,"[우리나라, 재난, 관리, 활동, 영향, 외, 요인, 다중, 응답, 분석, 소방, ..."
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...,"[연구자, 국가, 통합, 재난, 정체, 연구, 목적, 소방, 방재, 청과, 지방자치..."


# Rhino 분석 (명사)

In [ ]:
import rhinoMorph
rn = rhinoMorph.startRhino()

sentence['tokenized_nn'] = sentence['문장'].apply(lambda x : rhinoMorph.onlyMorph_list(rn, x, pos=['NNG', 'NNP', 'NP']))
sentence.head()

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


,태그,문장,tokenized_nn
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...,"[재난, 관리, 체계, 개선, 방법, 가지, 재난, 관리, 분야, 사실, 중추적, ..."
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...,"[연구, 재난, 관리, 조직, 부처, 간, 얼마, 재난, 관리, 관련, 공조, 체계..."
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...,"[연구, 표본, 특성, 설문지, 응답자, 근무처, 소방, 방재, 청, 지방자치단체]"
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...,"[우리나라, 재난, 관리, 활동, 영향, 외, 요인, 다중, 응답, 분석, 소방, ..."
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...,"[연구자, 국가, 통합, 재난, 정체, 연구, 목적, 소방, 방재, 청과, 지방자치..."


In [ ]:
# 전체 코퍼스의 상/하위 빈도수 토큰 출력 (tokenized1)
from collections import Counter

whole_words = np.hstack(sentence['tokenized_nn'].values)
whole_words_count = Counter(whole_words)
len(whole_words_count.items())

23166

In [ ]:
print(whole_words_count.most_common()[:2000])       # 상위 2000 개의 명사
# print(whole_words_count.most_common()[:-15001:-1])       # 하위 2000 개의 명사

[('연구', 21881), ('결과', 10375), ('분석', 7176), ('방법', 6201), ('영향', 5400), ('이', 5198), ('실험', 4916), ('특성', 4825), ('경우', 4563), ('자료', 4419), ('대상', 4399), ('후', 4042), ('효과', 3745), ('논문', 3473), ('데이터', 3425), ('시간', 3371), ('시스템', 3211), ('모델', 3174), ('변화', 3156), ('값', 3115), ('정보', 2726), ('평가', 2676), ('조사', 2655), ('차이', 2623), ('때', 2620), ('이상', 2619), ('요인', 2493), ('성능', 2443), ('측정', 2408), ('기반', 2405), ('다양', 2380), ('환경', 2317), ('지역', 2313), ('프로그램', 2311), ('군', 2270), ('환자', 2248), ('교육', 2247), ('개발', 2189), ('시험', 2164), ('사용', 2162), ('모형', 2152), ('기존', 2119), ('해석', 2032), ('이용', 2028), ('기준', 2013), ('과정', 1987), ('필요', 1944), ('조건', 1937), ('비교', 1932), ('관련', 1929), ('기술', 1919), ('온도', 1875), ('평균', 1865), ('관리', 1864), ('량', 1847), ('농도', 1830), ('기법', 1826), ('수준', 1818), ('정도', 1815), ('적용', 1761), ('향후', 1759), ('구조', 1756), ('처리', 1755), ('상태', 1752), ('치료', 1745), ('점', 1737), ('시', 1718), ('화재', 1700), ('설계', 1664), ('발생', 1616), ('학생', 1587), ('변수', 

In [ ]:
# 빈도수 높은 명사 리스트 만들기
common_list = []
common_tuple = whole_words_count.most_common()[:2000]

for item in common_tuple:
  if len(item[0])>1:
    common_list.append(item[0])
common_list

['연구',
 '결과',
 '분석',
 '방법',
 '영향',
 '실험',
 '특성',
 '경우',
 '자료',
 '대상',
 '효과',
 '논문',
 '데이터',
 '시간',
 '시스템',
 '모델',
 '변화',
 '정보',
 '평가',
 '조사',
 '차이',
 '이상',
 '요인',
 '성능',
 '측정',
 '기반',
 '다양',
 '환경',
 '지역',
 '프로그램',
 '환자',
 '교육',
 '개발',
 '시험',
 '사용',
 '모형',
 '기존',
 '해석',
 '이용',
 '기준',
 '과정',
 '필요',
 '조건',
 '비교',
 '관련',
 '기술',
 '온도',
 '평균',
 '관리',
 '농도',
 '기법',
 '수준',
 '정도',
 '적용',
 '향후',
 '구조',
 '처리',
 '상태',
 '치료',
 '화재',
 '설계',
 '발생',
 '학생',
 '변수',
 '방식',
 '세포',
 '관계',
 '안전',
 '활성',
 '건강',
 '기능',
 '학습',
 '문제',
 '영역',
 '알고리즘',
 '영상',
 '추출물',
 '요소',
 '실제',
 '크기',
 '활용',
 '증가',
 '목적',
 '향상',
 '예측',
 '분포',
 '반응',
 '인식',
 '물질',
 '형태',
 '위치',
 '시료',
 '가능',
 '조직',
 '국내',
 '대상자',
 '유의',
 '능력',
 '개선',
 '단계',
 '방향',
 '감소',
 '교사',
 '공간',
 '스트레스',
 '방안',
 '위험',
 '신호',
 '바탕',
 '활동',
 '행동',
 '기초',
 '속도',
 '서비스',
 '범위',
 '일반적',
 '다음',
 '센서',
 '검증',
 '시뮬레이션',
 '도구',
 '내용',
 '공정',
 '함량',
 '수행',
 '소재',
 '검사',
 '표면',
 '가능성',
 '기간',
 '최대',
 '전체',
 '설문',
 '집단',
 '효과적',
 '통계적',
 '현장',
 '분류',
 '동안',
 '상황',
 '

# Rhino 분석 (영어단어)

In [ ]:
import rhinoMorph
rn = rhinoMorph.startRhino()

sentence['tokenized_f'] = sentence['문장'].apply(lambda x : rhinoMorph.onlyMorph_list(rn, x, pos=['SL']))
sentence.head()

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


,태그,문장,tokenized_nn,tokenized_f
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...,"[재난, 관리, 체계, 개선, 방법, 가지, 재난, 관리, 분야, 사실, 중추적, ...",[]
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...,"[연구, 재난, 관리, 조직, 부처, 간, 얼마, 재난, 관리, 관련, 공조, 체계...",[]
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...,"[연구, 표본, 특성, 설문지, 응답자, 근무처, 소방, 방재, 청, 지방자치단체]",[TABLE]
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...,"[우리나라, 재난, 관리, 활동, 영향, 외, 요인, 다중, 응답, 분석, 소방, ...","[GT, GT, GT, GT, GT, GT]"
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...,"[연구자, 국가, 통합, 재난, 정체, 연구, 목적, 소방, 방재, 청과, 지방자치...",[]


In [ ]:
# 전체 코퍼스의 하위 빈도수 토큰 출력 (tokenized1)
from collections import Counter

whole_words = np.hstack(sentence['tokenized_f'].values)
whole_words_count = Counter(whole_words)
print(whole_words_count.most_common()[:1000])       # 상위 1000 개의 영어단어
print(whole_words_count.most_common()[:-1001:-1])       # 하위 1000개의 영어단어

[('M', 1703), ('C', 951), ('FIG', 789), ('A', 703), ('D', 673), ('H', 672), ('B', 654), ('MM', 637), ('O', 598), ('SPSS', 579), ('TABLE', 577), ('S', 538), ('G', 527), ('TEST', 505), ('T', 477), ('K', 468), ('P', 459), ('X', 373), ('L', 371), ('N', 355), ('V', 340), ('AL', 339), ('TTEST', 328), ('PH', 313), ('NM', 306), ('CM', 296), ('ACID', 296), ('ANOVA', 295), ('R', 288), ('CO', 260), ('E', 249), ('GML', 249), ('ET', 230), ('FIGURE', 228), ('F', 223), ('AND', 216), ('IL', 212), ('DB', 210), ('KIM', 203), ('DNA', 202), ('ML', 198), ('KM', 189), ('Y', 176), ('MS', 173), ('CASE', 169), ('NO', 164), ('CT', 159), ('PLT', 159), ('LT', 156), ('WT', 156), ('OC', 149), ('LEE', 149), ('W', 147), ('MULTIPLE', 147), ('MG', 147), ('LPS', 146), ('IOT', 145), ('TYPE', 141), ('DPPH', 138), ('OF', 138), ('RAW', 138), ('I', 134), ('GT', 132), ('FE', 132), ('IN', 122), ('ISO', 121), ('CU', 121), ('MODEL', 120), ('CELL', 120), ('MGKG', 120), ('RANGE', 118), ('LED', 117), ('MPA', 116), ('FOR', 115), ('P

In [ ]:
# 영어단어 토큰의 총 개수
len(whole_words_count.items())

14801

In [ ]:
# 빈도수 높은 영어단어 리스트 만들기
common_list_en = []
common_tuple = whole_words_count.most_common()[:1000]

for item in common_tuple:
  if len(item[0])>1:
    common_list_en.append(item[0])
common_list_en

['FIG',
 'MM',
 'SPSS',
 'TABLE',
 'TEST',
 'AL',
 'TTEST',
 'PH',
 'NM',
 'CM',
 'ACID',
 'ANOVA',
 'CO',
 'GML',
 'ET',
 'FIGURE',
 'AND',
 'IL',
 'DB',
 'KIM',
 'DNA',
 'ML',
 'KM',
 'MS',
 'CASE',
 'NO',
 'CT',
 'PLT',
 'LT',
 'WT',
 'OC',
 'LEE',
 'MULTIPLE',
 'MG',
 'LPS',
 'IOT',
 'TYPE',
 'DPPH',
 'OF',
 'RAW',
 'GT',
 'FE',
 'IN',
 'ISO',
 'CU',
 'MODEL',
 'CELL',
 'MGKG',
 'RANGE',
 'LED',
 'MPA',
 'FOR',
 'PM',
 'TIO',
 'KS',
 'RADICAL',
 'STATISTICS',
 'INDEX',
 'APOPTOSIS',
 'KG',
 'SYSTEM',
 'ONEWAY',
 'HZ',
 'ICT',
 'KW',
 'ASSAY',
 'DUNCANS',
 'VER',
 'SP',
 'PPM',
 'IBM',
 'API',
 'PARK',
 'CYTOKINE',
 'SAS',
 'VERSION',
 'GHZ',
 'MGML',
 'ESS',
 'CORRELATION',
 'GPS',
 'METHOD',
 'AR',
 'MRNA',
 'CD',
 'CA',
 'STUDENTS',
 'FDS',
 'DUNCAN',
 'SW',
 'SCALE',
 'IT',
 'KIT',
 'HPLC',
 'RPM',
 'DATA',
 'GROUP',
 'TH',
 'CASPASE',
 'SI',
 'CFD',
 'ANALYSIS',
 'KMH',
 'TYROSINASE',
 'PEARSON',
 'XRD',
 'TNF',
 'PLATE',
 'WINDOWS',
 'GA',
 'DC',
 'MHZ',
 'CH',
 'AC',
 'TOTAL'

In [ ]:
common_list_en.append(str.upper('resources'))
common_list_en.append(str.upper('database'))

In [ ]:
# 리스트를 텍스트로 저장하기
def write_txt(list, fname, sep) :
    file = open(fname, 'w')
    vstr = ''
    for a in list :
        vstr = vstr + str(a) + sep
    vstr = vstr.rstrip(sep)  # 마지막에도 추가되는  sep을 삭제 
    file.writelines(vstr)      # 한 라인씩 저장 
    file.close()
    print('[ 파일 저장 완료 ]')

In [ ]:
write_txt(common_list_en, '1000common_list_en.txt', ',')

[ 파일 저장 완료 ]


# 전체 데이터에 함수 적용위해 새롭게 데이터 불러오기 

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/aiContest/20201015_경진대회데이터_문장의미태깅.xlsx', sheet_name='문장f')
df.head()

,논문고유번호,태그,문장,대표여부
0,JAKO201003949970469,문제 정의,재난관리체계의 개선(일원화)방법으로는 여러 가지\r\n가 있을 수 있겠으나 여기서는...,Y
1,JAKO201003949970469,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간\r\n의 얼마만큼 재난관리와 관련...,N
2,JAKO201003949970469,대상 데이터,본 연구의 사용된 표본의 특성은 Table 5와 같다. 설\r\n문지 응답자들의 근...,N
3,JAKO201003949970469,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요\r\n인을 다중응답 분석을 이용해서...,N
4,JAKO201003949970469,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구\r\n목적을 달성하기 위하여 소방방재...,Y


# 결측값 전처리

In [ ]:
# 문장 '없음' 인덱스 리스트로 생성
non_list = list(df[df['문장']=='없음'].index)+list(df[df['문장'].str.get(-2)=='없'].index)
# 제거 전 행 개수 출력
print(len(df))
# 값이 없는 행 제거
df = df.drop([13229, 13230, 13237, 13244]+non_list, axis=0)
print(len(df))
# 중복제거
df.drop_duplicates(subset = ['태그','문장'], inplace=True)
print(len(df))

63436
63420
63319


# 컬럼 / 문장 전처리

In [ ]:
# 필요없는 열 제거
df = df.drop(['논문고유번호', '대표여부'], axis=1)
df.head()

,태그,문장
0,문제 정의,재난관리체계의 개선(일원화)방법으로는 여러 가지\r\n가 있을 수 있겠으나 여기서는...
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간\r\n의 얼마만큼 재난관리와 관련...
2,대상 데이터,본 연구의 사용된 표본의 특성은 Table 5와 같다. 설\r\n문지 응답자들의 근...
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요\r\n인을 다중응답 분석을 이용해서...
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구\r\n목적을 달성하기 위하여 소방방재...


In [ ]:
# 필요없는 문자 제거
sentence = df
sentence = sentence.replace({'\r':'','\n':'','\([^)]*\)' : ''}, regex=True)
sentence['문장'] = sentence['문장'].str.upper()
sentence['문장'] = sentence['문장'].str.replace("[^가-힣A-Z0-9=%\. ]","")
sentence.head()

,태그,문장
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...


In [ ]:
df = sentence
df

,태그,문장
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...
...,...,...
63431,기술 정의,스마트시티는 일반적으로 물리적 도시시설이 사물인터넷 등 ICT 기술과 접목되어 효율...
63432,제안 방법,본 연구에서는 스마트시티에 대한 국내외 현황을 조사하고 리빙랩 방식으로 시민들의 생...
63433,대상 데이터,국내외 스마트시티 현황 및 추진사례에서 발현된 핵심전략과 특화프로그램을 종합하여 생...
63434,성능/효과,이에 따른 주요 실행계획으로는 도시 스마트시스템을 구축하여 건강하고 살기 좋은 도시...


# 전체 데이터에 단어제거 함수 적용 (영어단어에만 적용)

In [ ]:
# 단어제거 함수

'''
각 문장의 형태소분석결과가 영어인 것 중 중요 단어가 아닌 것은 지운다.
'''

def del_unimportant_n(sentence,common_n_list,rn):
  del_dic = {}
  n_list = rhinoMorph.onlyMorph_list(rn, sentence, pos=['SL'])
  for n in n_list:
    if len(n)>1:
      if n not in common_n_list:
        del_dic[n] = ''
  if len(del_dic)!=0:
    sentence = pd.Series(sentence)
    new_sentence = sentence.replace(del_dic, regex=True)
    list(new_sentence[0])
  else:
    new_sentence = [sentence]
  return new_sentence

In [ ]:
df['new_sent'] = 0
df

,태그,문장,new_sent
0,문제 정의,재난관리체계의 개선방법으로는 여러 가지가 있을 수 있겠으나 여기서는 재난관리분야에서...,0
1,문제 정의,이 연구는 재난관리에 관여하는 조직 및 부처들 간의 얼마만큼 재난관리와 관련한 공조...,0
2,대상 데이터,본 연구의 사용된 표본의 특성은 TABLE 5와 같다. 설문지 응답자들의 근무처를 ...,0
3,대상 데이터,우리나라에서 재난관리 활동에 영향을 주는 외적요인을 다중응답 분석을 이용해서 살펴보...,0
4,대상 데이터,연구자는 국가통합재난관리행정체계에 대한 연구목적을 달성하기 위하여 소방방재청과 지...,0
...,...,...,...
63431,기술 정의,스마트시티는 일반적으로 물리적 도시시설이 사물인터넷 등 ICT 기술과 접목되어 효율...,0
63432,제안 방법,본 연구에서는 스마트시티에 대한 국내외 현황을 조사하고 리빙랩 방식으로 시민들의 생...,0
63433,대상 데이터,국내외 스마트시티 현황 및 추진사례에서 발현된 핵심전략과 특화프로그램을 종합하여 생...,0
63434,성능/효과,이에 따른 주요 실행계획으로는 도시 스마트시스템을 구축하여 건강하고 살기 좋은 도시...,0


In [ ]:
# 전체 데이터프레임에 단어제거 함수 적용
for i in df.index:
  df['new_sent'][i] = del_unimportant_n(df['문장'][i],common_list_en,rn)[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# 확인 및 전처리 마무리

In [ ]:
df[df.index==63424]

,태그,문장,new_sent
63424,데이터처리,CHATTERJEE와 HADI 와 FARAWAY 를 참조하여 독립성 검증은 DURB...,와 와 를 참조하여 독립성 검증은 통계량을 이용하고 정규성 검증은 통계량...


In [ ]:
# 필요없는 열 제거
df = df.drop(['문장'], axis=1)
df.tail(50)

,태그,new_sent
63386,성능/효과,특히 모형은 다른 경쟁모형과 비교할 때 가장 우수한 예측성능을 보여준다.
63387,성능/효과,결과적으로 이변량 환율자료예측에 있어 다변량 지수평활모형이 경쟁모형에 비하여 상대적...
63388,후속연구,이를 통하여 다음과 같은 3가지 점들은 흥미로운 결과이며 이는 추후 좀더 면밀한 검...
63389,문제 정의,따라서 시간의 흐름에 따라 관측된 자료의 특성에 맞는 결측값을 추정하는 방법을 사용...
63390,기술 정의,마이크로어레이 유전자 실험을 시간의 흐름에 따라서 연속적으로 수행되면서 얻어지는 자...
63391,제안 방법,본 논문에서 제안한 방법과 기존의 ANN 방법의 성능을 비교하기 위해 두 가지의 ...
63392,제안 방법,두 가지의 유사성 거리와 패턴일치지수를 이용해 결측값을 대체하는 방법인 방법을 제...
63393,제안 방법,결측값의 위치에 따라 이웃의 개수를 고정하지 않고 유동적으로 조정할 수 있는 ANN...
63394,제안 방법,본 논문에서는 시간경로 유전자 발현 자료에 대해서 전처리 과정으로 결측값을 추정하는...
63395,제안 방법,새로 제안한 방법은 결측값의 위치에 따른 이웃의 개수를 조정할 수 있는 ANN 방...


In [ ]:
# 전처리 결과 저장
df.to_csv("aicontest_data10.csv", mode='w', encoding='utf-8', index=False)

# 전처리된 데이터 불러와서 확인

In [ ]:
# 전처리된 데이터 불러오기 (앞의 전처리 과정 생략가능)
df = pd.read_csv('/content/aicontest_data10.csv')
print(len(df))
df.tail()

In [ ]:
df.sample(40)